In [129]:
import os
import pandas as pd
import numpy as np
import datetime as dt
import sys
import time

import preprocessing
import models
import prediction

from sklearn.model_selection import TimeSeriesSplit

In [16]:
path_data = '/Users/sergigomezpalleja/Downloads/'

In [39]:
train = pd.read_csv(path_data + 'raw_data_master.csv')

In [40]:
train.shape

(99456, 13)

In [41]:
list(train)

['Cluster',
 'Brand_Group',
 'Country',
 'date',
 'sales1',
 'sales2',
 'inv1',
 'inv2',
 'inv3',
 'inv4',
 'inv5',
 'inv6',
 'id']

In [42]:
train.head(n=1)

,Cluster,Brand_Group,Country,date,sales1,sales2,inv1,inv2,inv3,inv4,inv5,inv6,id
0,Cluster 1,Brand Group 12,Country 10,2012-01-01,44.214973,41.311052,NaN,NaN,0.0,0.0,NaN,NaN,1


In [52]:
df = train.copy()

In [53]:
df['Country'] = df['Country'].str.extract('(\d+)').astype(int)
df['Brand_Group'] = df['Brand_Group'].str.extract('(\d+)').astype(int)
df['Cluster'] = df['Cluster'].str.extract('(\d+)').astype(int)

/Users/sergigomezpalleja/anaconda3/envs/kaggle/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  if __name__ == '__main__':
/Users/sergigomezpalleja/anaconda3/envs/kaggle/lib/python3.6/site-packages/ipykernel/__main__.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  from ipykernel import kernelapp as app
/Users/sergigomezpalleja/anaconda3/envs/kaggle/lib/python3.6/site-packages/ipykernel/__main__.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  app.launch_new_instance()


In [54]:
df.head()

,Cluster,Brand_Group,Country,date,sales1,sales2,inv1,inv2,inv3,inv4,inv5,inv6,id
0,1,12,10,2012-01-01,44.214973,41.311052,NaN,NaN,0.000000,0.000000,NaN,NaN,1
1,1,12,10,2012-02-01,104.918692,95.697737,NaN,NaN,-3.441472,0.000000,NaN,NaN,2
2,1,12,10,2012-03-01,25.813126,21.010914,NaN,NaN,0.000000,0.000000,NaN,NaN,3
3,1,12,10,2012-04-01,21.735670,19.227058,NaN,NaN,0.000000,0.000000,NaN,NaN,4
4,1,12,10,2012-05-01,17.258679,15.390803,NaN,NaN,0.000000,-0.010893,NaN,NaN,5


In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99456 entries, 0 to 99455
Data columns (total 13 columns):
Cluster        99456 non-null int64
Brand_Group    99456 non-null int64
Country        99456 non-null int64
date           99456 non-null object
sales1         74520 non-null float64
sales2         77688 non-null float64
inv1           40236 non-null float64
inv2           40488 non-null float64
inv3           13440 non-null float64
inv4           50148 non-null float64
inv5           420 non-null float64
inv6           32844 non-null float64
id             99456 non-null int64
dtypes: float64(8), int64(4), object(1)
memory usage: 9.9+ MB


In [113]:
train = pd.read_csv(path_data + 'train_preprocessed_sergi_1.csv')

In [114]:
df = train.copy()

In [115]:
df['date'] = df['date'].astype('datetime64[ns]')

In [116]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99456 entries, 0 to 99455
Data columns (total 13 columns):
Cluster        99456 non-null int64
Brand_Group    99456 non-null int64
Country        99456 non-null int64
date           99456 non-null datetime64[ns]
sales1         74520 non-null float64
sales2         77688 non-null float64
inv1           40236 non-null float64
inv2           40488 non-null float64
inv3           13440 non-null float64
inv4           50148 non-null float64
inv5           420 non-null float64
inv6           32844 non-null float64
id             99456 non-null int64
dtypes: datetime64[ns](1), float64(8), int64(4)
memory usage: 9.9 MB


In [109]:
print("Percentage Missings Inv 1 ", df.loc[df.inv1.isnull()].shape[0]/df.shape[0])
print("Percentage Missings Inv 2 ", df.loc[df.inv2.isnull()].shape[0]/df.shape[0])
print("Percentage Missings Inv 3 ", df.loc[df.inv3.isnull()].shape[0]/df.shape[0])
print("Percentage Missings Inv 4 ", df.loc[df.inv4.isnull()].shape[0]/df.shape[0])
print("Percentage Missings Inv 5 ", df.loc[df.inv5.isnull()].shape[0]/df.shape[0])
print("Percentage Missings Inv 6 ", df.loc[df.inv6.isnull()].shape[0]/df.shape[0])

Percentage Missings Inv 1  0.5954391891891891
Percentage Missings Inv 2  0.5929054054054054
Percentage Missings Inv 3  0.8648648648648649
Percentage Missings Inv 4  0.49577702702702703
Percentage Missings Inv 5  0.995777027027027
Percentage Missings Inv 6  0.6697635135135135


In [119]:
df.loc[(df.inv5.isnull()) & (df.inv5 != 0)]

,Cluster,Brand_Group,Country,date,sales1,sales2,inv1,inv2,inv3,inv4,inv5,inv6,id


In [117]:
df['inv5'].fillna(value = 0.0, inplace = True)

In [126]:
df = pd.read_csv(path_data + 'train_preprocessed_sergi_1.csv')

In [138]:
train_X = df.loc[df.date < '2018-01-01']

In [139]:
values = train_X.date.unique()

In [145]:
len(values)

72

In [153]:
tscv = TimeSeriesSplit(n_splits = 5)

In [157]:
# iterating over folds, train model on each, forecast and calculate error
for train, val in tscv.split(values):
    print('Train: %s, Validation: %s' % (len(train), len(val)))
    dates_train = values[train]
    dates_val = values[val]
    print('Train: %s, Validation: %s' % (dates_train, dates_val))

Train: 12, Validation: 12
Train: ['2012-01-01' '2012-02-01' '2012-03-01' '2012-04-01' '2012-05-01'
 '2012-06-01' '2012-07-01' '2012-08-01' '2012-09-01' '2012-10-01'
 '2012-11-01' '2012-12-01'], Validation: ['2013-01-01' '2013-02-01' '2013-03-01' '2013-04-01' '2013-05-01'
 '2013-06-01' '2013-07-01' '2013-08-01' '2013-09-01' '2013-10-01'
 '2013-11-01' '2013-12-01']
Train: 24, Validation: 12
Train: ['2012-01-01' '2012-02-01' '2012-03-01' '2012-04-01' '2012-05-01'
 '2012-06-01' '2012-07-01' '2012-08-01' '2012-09-01' '2012-10-01'
 '2012-11-01' '2012-12-01' '2013-01-01' '2013-02-01' '2013-03-01'
 '2013-04-01' '2013-05-01' '2013-06-01' '2013-07-01' '2013-08-01'
 '2013-09-01' '2013-10-01' '2013-11-01' '2013-12-01'], Validation: ['2014-01-01' '2014-02-01' '2014-03-01' '2014-04-01' '2014-05-01'
 '2014-06-01' '2014-07-01' '2014-08-01' '2014-09-01' '2014-10-01'
 '2014-11-01' '2014-12-01']
Train: 36, Validation: 12
Train: ['2012-01-01' '2012-02-01' '2012-03-01' '2012-04-01' '2012-05-01'
 '2012-06-0